# 04 SynthSR, SynthSeg, Synthetic White matter lobes using KNN


Arvid Lundervold/Marianne Hannisdal

Last updated: **2025-03-13**  

Kernel: segment_glioma (Python 3.9.19)

In [1]:
!nvidia-smi

Thu Mar 13 18:53:09 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.06             Driver Version: 535.183.06   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX 3500 Ada Gene...    On  | 00000000:01:00.0 Off |                  Off |
| N/A   43C    P0              N/A /  90W |      8MiB / 12282MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Import libraries we will use

In [1]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '18'
home = os.path.expanduser('~')
import os.path as op
import glob
import shutil
import subprocess as subp
import pathlib
import platform
import shutil
import IPython
from datetime import date
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import nibabel as nib
from nibabel.viewers import OrthoSlicer3D
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import pickle
import pydicom
from pydicom import dcmread
# import imageio
import imageio.v2 as imageio

%matplotlib inline
home = os.path.expanduser('~')            # To make a path to local home directory
                       
warnings.filterwarnings('ignore')  # To ignore warnings 
  

In [4]:
import tensorflow as tf

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

# Check if GPU is available
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print("GPU available:", gpu_devices)
else:
    print("No GPU found. Running on CPU.")


2025-03-13 18:53:34.232007: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-13 18:53:34.239458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-13 18:53:34.247991: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-13 18:53:34.250529: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 18:53:34.257238: I tensorflow/core/platform/cpu_feature_guar

TensorFlow version: 2.17.0
No GPU found. Running on CPU.


I0000 00:00:1741888416.438768   54061 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-13 18:53:36.456299: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
print(f'platform: {platform.platform()}')
print(f'python version: {platform.python_version()}')
print(f'pydicom version: {pydicom.__version__}')
print(f'nibabel version: {nib.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')


platform: Linux-6.8.0-45-generic-x86_64-with-glibc2.39
python version: 3.11.9
pydicom version: 2.4.4
nibabel version: 5.2.1
pandas version: 2.2.2
numpy version: 1.26.4


In [8]:
verbose = True

**Defining directories:**

In [ ]:
PRJ_DIR = f'{home}/prj/glioma_recurrence/glioma_recurrence'
SITE = '10'

REGISTERED_dir = f'{PRJ_DIR}/data/{SITE}/registered_1'
SYNTHSEG_dir = f'{REGISTERED_dir}/synthseg'
SYNTH_SR_dir = f'{REGISTERED_dir}/synth_sr'
FS741_dir = f'{PRJ_DIR}/data/{SITE}/FS-741'

**Define all the channels and get a list of all (test) subjects in the directory:**

In [12]:
#all_chns = ['T1', 'CT1', 'T2', 'FLAIR']
all_chns = ['CT1']

all_dirs = sorted(glob.glob(f'{REGISTERED_dir}/{SITE}_*'))    # exams from local recordings
subjs = [ os.path.basename(d) for d in all_dirs]
n = len(subjs)

print(f'{n} examinations:')
print('[')
for i, sub in enumerate(subjs):
      print(f"'{sub}',")
print(']')

1 examinations:
[
'10_036',
]


In [13]:
# Execute tree command on the easyreg subdirecory 
print(os.popen(f'{TREE} {REGISTERED_dir}').read())

/home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered_1
└── 10_036

1 directory, 0 files



### Running Freesurfer's `mri_synthSR` and `mri_synthseg` on the coregistred T1 (CT1) images in the `REGISTERED` directory

See also https://surfer.nmr.mgh.harvard.edu/fswiki/WMH-SynthSeg

`mri_WMHsynthseg --i <input> --o <output> [--csv_vols <CSV file>] [--device <device>]  [--threads <threads>] [--crop] [--save_lesion_probabilities]`

In [32]:
!which mri_synthseg

/usr/local/freesurfer/bin/mri_synthseg


In [14]:
# Get the sorted list of directories (exams) inside EASYREG_dir
exams_list = sorted([x for x in next(os.walk(REGISTERED_dir))[1]])

# Convert the list to a string with each element inside single quotes and separated by a space
exams_list_str = " ".join([f"'{x}'" for x in exams_list])

# Set it as an environment variable so Bash can access it
%env EXAMS_LIST_STR=$exams_list_str


env: EXAMS_LIST_STR='10_036'


In [15]:
%%bash

# Access the environment variable created in Python
list=($EXAMS_LIST_STR)

# Removing commas and storing in a new array
list_new=()

for i in "${!list[@]}"; do
    list_new[$i]=${list[$i]//,/}
done

# Printing the cleaned list
for element in "${list_new[@]}"; do
    echo "$element"
done


'10_036'


In [16]:
%env REGISTERED_dir=$REGISTERED_dir

env: REGISTERED_dir=/home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered_1


In [17]:
# Get the sorted list of directories (exams) inside REGISTERED_dir
exams_list = sorted([x for x in next(os.walk(REGISTERED_dir))[1]])

# Convert the list to a string with each element inside single quotes and separated by a space
exams_list_str = " ".join([f"'{x}'" for x in exams_list])

# Set it as an environment variable so Bash can access it
%env EXAMS_LIST_STR=$exams_list_str

env: EXAMS_LIST_STR='10_036'


## performing SynthSR and SynthSeg on CT1

In [18]:
%%bash -s "$REGISTERED_dir"

echo "REGISTERED directory: $1"

# Set up environment
FREESURFER_HOME=/usr/local/freesurfer/; export FREESURFER_HOME
PATH=${FREESURFER_HOME}/bin:${PATH}; export PATH
FSLDIR=/usr/local/fsl; export FSLDIR
PATH=${FSLDIR}/bin:${PATH}; export PATH
. ${FSLDIR}/etc/fslconf/fsl.sh
source ${FREESURFER_HOME}/SetUpFreeSurfer.sh

# Convert the space-separated string of quoted elements into an array
IFS=' ' read -r -a list_new <<< "$EXAMS_LIST_STR"

# Process each subject folder
for subject_folder in "${list_new[@]}"; do
    # Remove any surrounding single quotes from the folder name
    subject_folder=${subject_folder//\'/}
    echo "PROCESSING SUBJECT FOLDER: $subject_folder"
    
    # Find the primary scan date by looking at the CT1 file pattern
    primary_scan=$(find "$1/$subject_folder" -maxdepth 1 -type f -name "${subject_folder}_*_CT1_coreg.nii.gz" | head -n 1)
    
    if [ -z "$primary_scan" ]; then
        echo "ERROR: No CT1 scan found for $subject_folder"
        continue
    fi
    
    # Extract the primary date from the filename
    primary_date=$(basename "$primary_scan" | cut -d'_' -f3)
    echo "Primary scan date: $primary_date"
    
    # Process primary scan
    primary_CT1="${1}/${subject_folder}/${subject_folder}_${primary_date}_CT1_coreg.nii.gz"
    
    if [ ! -f "$primary_CT1" ]; then
        echo "ERROR: CT1 not available for primary scan"
        continue
    fi
    
    # Process primary scan with SynthSR
    echo "Running SynthSR on primary scan"
    mri_synthsr \
        --i "$primary_CT1" \
        --o "${1}/${subject_folder}/${subject_folder}_${primary_date}_CT1_SynthSR.nii.gz" \
        --threads 18
        
    # Run SynthSeg on primary SynthSR output
    if [ -f "${1}/${subject_folder}/${subject_folder}_${primary_date}_CT1_SynthSR.nii.gz" ]; then
        echo "Running SynthSeg on primary SynthSR output"
        mri_synthseg \
            --i "${1}/${subject_folder}/${subject_folder}_${primary_date}_CT1_SynthSR.nii.gz" \
            --o "${1}/${subject_folder}/${subject_folder}_${primary_date}_CT1_SynthSR_synthseg_parc_robust.nii.gz" \
            --vol "${1}/${subject_folder}/${subject_folder}_${primary_date}_CT1_SynthSR_synthseg_parc_robust_vol.csv" \
            --parc \
            --robust \
            --threads 19
    fi
    
    # Find and process recurrent scan
    recurrent_CT1_path=$(find "$1/$subject_folder" -maxdepth 1 -type f -name "${subject_folder}_*_CT1_coreg_registered_to_${subject_folder}_${primary_date}.nii.gz" | head -n 1)
    
    if [ -n "$recurrent_CT1_path" ]; then
        # Extract recurrent date from filename
        recurrent_date=$(basename "$recurrent_CT1_path" | cut -d'_' -f3)
        echo "Processing recurrent scan from date: $recurrent_date"
        
        # Process recurrent scan with SynthSR
        echo "Running SynthSR on recurrent scan"
        mri_synthsr \
            --i "$recurrent_CT1_path" \
            --o "${1}/${subject_folder}/${subject_folder}_${recurrent_date}_CT1_SynthSR_registered_to_${subject_folder}_${primary_date}.nii.gz" \
            --threads 18
            
        # Run SynthSeg on recurrent SynthSR output
        if [ -f "${1}/${subject_folder}/${subject_folder}_${recurrent_date}_CT1_SynthSR_registered_to_${subject_folder}_${primary_date}.nii.gz" ]; then
            echo "Running SynthSeg on recurrent SynthSR output"
            mri_synthseg \
                --i "${1}/${subject_folder}/${subject_folder}_${recurrent_date}_CT1_SynthSR_registered_to_${subject_folder}_${primary_date}.nii.gz" \
                --o "${1}/${subject_folder}/${subject_folder}_${recurrent_date}_CT1_SynthSR_synthseg_parc_robust_registered_to_${subject_folder}_${primary_date}.nii.gz" \
                --vol "${1}/${subject_folder}/${subject_folder}_${recurrent_date}_CT1_SynthSR_synthseg_parc_robust_registered_to_${subject_folder}_${primary_date}_vol.csv" \
                --parc \
                --robust \
                --threads 19
        fi
    else
        echo "No recurrent CT1 scan found for $subject_folder"
    fi
    
    echo "Completed processing for $subject_folder"
done

REGISTERED directory: /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered_1
PROCESSING SUBJECT FOLDER: 10_036
Primary scan date: 20150101
Running SynthSR on primary scan
Using general model from January 2023 (version 2)
/usr/local/freesurfer/models/synthsr_v20_230130.h5
using 18 threads
predicting 1/1
Prediction without flipping
1/1 [==============================] - 4s 4s/step
Prediction with flipping
1/1 [==============================] - 4s 4s/step

prediction  saved in: /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered_1/10_036/10_036_20150101_CT1_SynthSR.nii.gz

If you use this tool in a publication, please cite:


Joint super-resolution and synthesis of 1 mm isotropic MP-RAGE volumes from clinical 
MRI exams with scans of different orientation, resolution and contrast
JE Iglesias, B Billot, Y Balbastre, A Tabari, J Conklin, RG Gonzalez, DC Alexander,
P Golland, BL Edlow, B Fischl, for the ADNI
NeuroImage, 118206 (2021)



SynthSR: a p

## Computing synthetic white-matter lobes based on KNN (K=3) of synthseg-derived hemispheric white matter 

In [19]:
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

# Get the subject list from environment variable and clean it
subject_list_str = os.environ['EXAMS_LIST_STR']
subject_list = [s.strip("'") for s in subject_list_str.split()]

print("Processing subjects:", subject_list)  # For verification

LOBE_MAPPING = {
    # Left hemisphere cortical lobes
    'left_frontal': [
        1002,  # ctx-lh-caudalanteriorcingulate
        1003,  # ctx-lh-caudalmiddlefrontal
        1012,  # ctx-lh-lateralorbitofrontal
        1014,  # ctx-lh-medialorbitofrontal
        1018,  # ctx-lh-parsopercularis
        1019,  # ctx-lh-parsorbitalis
        1020,  # ctx-lh-parstriangularis
        1024,  # ctx-lh-precentral
        1026,  # ctx-lh-rostralanteriorcingulate
        1027,  # ctx-lh-rostralmiddlefrontal
        1028,  # ctx-lh-superiorfrontal
        1032   # ctx-lh-frontalpole
    ],

    'left_parietal': [
        1008,  # ctx-lh-inferiorparietal
        1022,  # ctx-lh-postcentral
        1025,  # ctx-lh-precuneus
        1029,  # ctx-lh-superiorparietal
        1031,  # ctx-lh-supramarginal
        1010   # ctx-lh-isthmuscingulate
    ],

    'left_temporal': [
        1001,  # ctx-lh-bankssts
        1006,  # ctx-lh-entorhinal
        1007,  # ctx-lh-fusiform
        1009,  # ctx-lh-inferiortemporal
        1015,  # ctx-lh-middletemporal
        1016,  # ctx-lh-parahippocampal
        1030,  # ctx-lh-superiortemporal
        1033,  # ctx-lh-temporalpole
        1034,  # ctx-lh-transversetemporal
        17,    # Left-Hippocampus
        18     # Left-Amygdala
    ],

    'left_occipital': [
        1005,  # ctx-lh-cuneus
        1011,  # ctx-lh-lateraloccipital
        1013,  # ctx-lh-lingual
        1021   # ctx-lh-pericalcarine
    ],

    # Right hemisphere cortical lobes
    'right_frontal': [
        2002,  # ctx-rh-caudalanteriorcingulate
        2003,  # ctx-rh-caudalmiddlefrontal
        2012,  # ctx-rh-lateralorbitofrontal
        2014,  # ctx-rh-medialorbitofrontal
        2018,  # ctx-rh-parsopercularis
        2019,  # ctx-rh-parsorbitalis
        2020,  # ctx-rh-parstriangularis
        2024,  # ctx-rh-precentral
        2026,  # ctx-rh-rostralanteriorcingulate
        2027,  # ctx-rh-rostralmiddlefrontal
        2028,  # ctx-rh-superiorfrontal
        2032   # ctx-rh-frontalpole
    ],

    'right_parietal': [
        2008,  # ctx-rh-inferiorparietal
        2022,  # ctx-rh-postcentral
        2025,  # ctx-rh-precuneus
        2029,  # ctx-rh-superiorparietal
        2031,  # ctx-rh-supramarginal
        2010   # ctx-rh-isthmuscingulate
    ],

    'right_temporal': [
        2001,  # ctx-rh-bankssts
        2006,  # ctx-rh-entorhinal
        2007,  # ctx-rh-fusiform
        2009,  # ctx-rh-inferiortemporal
        2015,  # ctx-rh-middletemporal
        2016,  # ctx-rh-parahippocampal
        2030,  # ctx-rh-superiortemporal
        2033,  # ctx-rh-temporalpole
        2034,  # ctx-rh-transversetemporal
        53,    # Right-Hippocampus
        54     # Right-Amygdala
    ],

    'right_occipital': [
        2005,  # ctx-rh-cuneus
        2011,  # ctx-rh-lateraloccipital
        2013,  # ctx-rh-lingual
        2021   # ctx-rh-pericalcarine
    ],

    # Non-lobular regions
    'brainstem': [16],  # Brain-Stem
    'corpus_callosum': [
        251,  # CC_Posterior
        252,  # CC_Mid_Posterior
        253,  # CC_Central
        254,  # CC_Mid_Anterior
        255   # CC_Anterior
    ]
}


# Define output labels for lobes - using values in the 900s range to avoid conflicts
LOBE_LABELS = {
    'left_frontal': 3201,
    'left_parietal': 3206,
    'left_temporal': 3205,
    'left_occipital': 3204,
    'right_frontal': 4201,
    'right_parietal': 4206,
    'right_temporal': 4205,
    'right_occipital': 4204
}

def get_voxel_coordinates(mask):
    """Get coordinates of all non-zero voxels in the mask."""
    return np.array(np.where(mask)).T

def process_voxel(args):
    """Process a single voxel for white matter segmentation with fixed K=3."""
    wm_coord, train_coords, train_labels = args
    # Find nearest neighbors
    dists = np.sqrt(np.sum((train_coords - wm_coord)**2, axis=1))
    
    # Get 3 nearest neighbors
    nearest_indices = np.argsort(dists)[:3]  # Fixed K=3
    nearest_labels = train_labels[nearest_indices]
    nearest_dists = dists[nearest_indices]
    
    # Weight by inverse distance
    weights = 1 / (nearest_dists + 1e-6)
    weights /= weights.sum()
    
    # Weighted vote
    label_votes = defaultdict(float)
    for label, weight in zip(nearest_labels, weights):
        label_votes[label] += weight
    
    # Get label with highest weighted vote
    predicted_label = max(label_votes.items(), key=lambda x: x[1])[0]
    return predicted_label

def segment_wm_lobes(input_path, output_path, n_jobs=20):
    """Segment white matter into lobes using vectorized KNN."""
    print(f"Processing {input_path}")
    
    # Load the SynthSeg parcellation
    img = nib.load(input_path)
    data = img.get_fdata()
    
    # Create output array
    output = np.zeros_like(data)
    
    # Process each hemisphere separately
    for hemi in ['left', 'right']:
        print(f"Processing {hemi} hemisphere")
        # Get WM mask for this hemisphere
        wm_label = 2 if hemi == 'left' else 41
        wm_mask = data == wm_label
        
        if not np.any(wm_mask):
            continue
        
        # Get coordinates of WM voxels
        wm_coords = get_voxel_coordinates(wm_mask)
        print(f"Found {len(wm_coords)} white matter voxels")
        
        # Prepare training data from cortical regions
        train_coords = []
        train_labels = []
        
        for lobe, label in LOBE_LABELS.items():
            if lobe.startswith(hemi):
                lobe_mask = np.zeros_like(data, dtype=bool)
                for region in LOBE_MAPPING[lobe]:
                    lobe_mask |= (data == region)
                
                if np.any(lobe_mask):
                    coords = get_voxel_coordinates(lobe_mask)
                    train_coords.extend(coords)
                    train_labels.extend([label] * len(coords))
        
        if train_coords:
            print(f"Found {len(train_coords)} training points")
            train_coords = np.array(train_coords)
            train_labels = np.array(train_labels)
            
            # Use scikit-learn's KNN directly - much faster than manual implementation
            print("Training KNN classifier")
            knn = KNeighborsClassifier(n_neighbors=3, weights='distance', n_jobs=n_jobs)
            knn.fit(train_coords, train_labels)
            
            # Predict in batches to avoid memory issues
            batch_size = 100000  # Adjust based on available memory
            n_batches = (len(wm_coords) + batch_size - 1) // batch_size
            
            print("Predicting labels")
            for i in tqdm(range(n_batches), desc=f"{hemi} hemisphere progress"):
                start_idx = i * batch_size
                end_idx = min((i + 1) * batch_size, len(wm_coords))
                batch_coords = wm_coords[start_idx:end_idx]
                
                # Predict for batch
                batch_predictions = knn.predict(batch_coords)
                
                # Assign predictions to output
                for coord, label in zip(batch_coords, batch_predictions):
                    output[tuple(coord)] = label
    
    # Save the result
    print(f"Saving output to {output_path}")
    nib_out = nib.Nifti1Image(output, img.affine, img.header)
    nib.save(nib_out, output_path)
    print("Processing complete")

def process_subject(args):
    """Process a single subject"""
    registered_dir, subject = args
    print(f"\nProcessing subject: {subject}")
    
    # Find primary and recurrent scans
    subject_dir = os.path.join(registered_dir, subject)
    
    primary_scan = None
    recurrent_scan = None
    
    # Updated file pattern matching for CT1
    for file in os.listdir(subject_dir):
        if file.endswith('.nii.gz'):
            if '_CT1_SynthSR_synthseg_parc_robust.nii.gz' in file and 'registered_to' not in file:
                primary_scan = file
            elif '_CT1_SynthSR_synthseg_parc_robust_registered_to' in file and subject in file:
                recurrent_scan = file
    
    if primary_scan:
        print(f"Processing primary scan: {primary_scan}")
        input_path = os.path.join(subject_dir, primary_scan)
        output_path = os.path.join(subject_dir, 
            primary_scan.replace('_CT1_SynthSR_synthseg_parc_robust.nii.gz', 
                               '_CT1_SR_synthetic_wm_lobes.nii.gz'))
        segment_wm_lobes(input_path, output_path, n_jobs=20)
    else:
        print(f"No primary CT1 scan found for subject {subject}")

    if recurrent_scan:
        print(f"Processing recurrent scan: {recurrent_scan}")
        input_path = os.path.join(subject_dir, recurrent_scan)
        output_path = os.path.join(subject_dir,
            recurrent_scan.replace('_CT1_SynthSR_synthseg_parc_robust_registered_to',
                                 '_CT1_SR_synthetic_wm_lobes_registered_to'))
        segment_wm_lobes(input_path, output_path, n_jobs=20)
    else:
        print(f"No recurrent CT1 scan found for subject {subject}")

def process_all_subjects(registered_dir, subject_list):
    """Process all subjects with progress bar"""
    print(f"Processing {len(subject_list)} subjects")
    
    for subject in tqdm(subject_list, desc="Overall progress", position=0):
        process_subject((registered_dir, subject))

# Execute the processing (either for one subject or all)
registered_dir = os.environ['REGISTERED_dir']

# For all subjects:
process_all_subjects(registered_dir, subject_list)
print("All processing complete")

Processing subjects: ['10_036']
Processing 1 subjects


Overall progress:   0%|          | 0/1 [00:00<?, ?it/s]


Processing subject: 10_036
Processing primary scan: 10_036_20150101_CT1_SynthSR_synthseg_parc_robust.nii.gz
Processing /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered_1/10_036/10_036_20150101_CT1_SynthSR_synthseg_parc_robust.nii.gz
Processing left hemisphere
Found 272608 white matter voxels
Found 294614 training points
Training KNN classifier
Predicting labels


left hemisphere progress: 100%|██████████| 3/3 [00:00<00:00,  3.13it/s]


Processing right hemisphere
Found 247298 white matter voxels
Found 290607 training points
Training KNN classifier
Predicting labels


...

Saving output to /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered_1/10_036/10_036_20150101_CT1_SR_synthetic_wm_lobes.nii.gz
Processing complete
Processing recurrent scan: 10_036_20160529_CT1_SynthSR_synthseg_parc_robust_registered_to_10_036_20150101.nii.gz
Processing /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered_1/10_036/10_036_20160529_CT1_SynthSR_synthseg_parc_robust_registered_to_10_036_20150101.nii.gz
Processing left hemisphere
Found 256022 white matter voxels
Found 282547 training points
Training KNN classifier
Predicting labels


left hemisphere progress: 100%|██████████| 3/3 [00:00<00:00,  3.85it/s]


Processing right hemisphere
Found 251015 white matter voxels
Found 292444 training points
Training KNN classifier
Predicting labels


Overall progress: 100%|██████████| 1/1 [00:05<00:00,  5.21s/it]

Saving output to /home/marianne/prj/glioma_recurrence/glioma_recurrence/data/10/registered_1/10_036/10_036_20160529_CT1_SR_synthetic_wm_lobes_registered_to_10_036_20150101.nii.gz
Processing complete
All processing complete
